In [4]:
import os
import csv
import numpy as np
import pprint
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, LSTM, BatchNormalization, TimeDistributed, GRU
from tensorflow.keras.activations import relu, softmax, sigmoid
# from tensorflow import lite

In [ ]:
CAPTURE_WINDOW = 2

### Sample time is 10ms ie sample rate = 1/10ms = 100hz
SAMPLE_RATE = 100

### Total samples per window
SAMPLES = CAPTURE_WINDOW * SAMPLE_RATE
FADE_LENGTH = 4

In [ ]:
def set_to_fixed_size(d):
    if len(d) >= SAMPLES:
        d = d[0: SAMPLES]
    elif len(d) < SAMPLES:
        base_index = len(d) - FADE_LENGTH
        for i in range(FADE_LENGTH):
            r = float(i) / (FADE_LENGTH - 1)
            d[base_index + i] = d[base_index + i] * (1 - r)
        d.extend([0] * (SAMPLES - len(d)))
    return d

In [ ]:
def load_data(file):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        data = []
        for i, row in enumerate(csv_reader):
            d = []
            ax = []
            ay = []
            az = []
            gx = []
            gy = []
            gz = []
            for i in range(0, len(row), 6):
                try:
                    ax.append(float(row[i + 0]))
                    ay.append(float(row[i + 1]))
                    az.append(float(row[i + 2]))
                    gx.append(float(row[i + 3]))
                    gy.append(float(row[i + 4]))
                    gz.append(float(row[i + 5]))
                except:
                    pass;
            # scale to fit into array of `SAMPLES`
            ax = set_to_fixed_size(ax)
            ay = set_to_fixed_size(ay)
            az = set_to_fixed_size(az)
            gx = set_to_fixed_size(gx)
            gy = set_to_fixed_size(gy)
            gz = set_to_fixed_size(gz)
            for i in range(SAMPLES):
                d.append([ax[i], ay[i], az[i], gx[i], gy[i], gz[i]])
            data.append(d)
    return data

In [ ]:
data_l = load_data('gesture.l.csv')
data_o = load_data('gesture.o.csv')
data_s = load_data('gesture.s.csv')

In [ ]:
c = int((len(data_o) * 2) / 3)
data_train_o = data_o[0: c]

print("Train data o: " + str(np.asarray(data_train_o).shape))

data_test_o = data_o[c: len(data_o)]

print.pprint("Test data o: " + str(np.asarray(data_test_o).shape))

c = int((len(data_s) * 2) / 3)
data_train_s = data_s[0: c]

print("Train data s: " + str(np.asarray(data_train_s).shape))

data_test_s = data_s[c: len(data_s)]

print("Test data s: " + str(np.asarray(data_test_s).shape))

In [ ]:
LABEL_L = 0
LABEL_O = 1
LABEL_S = 2

data_train = []
data_test = []

data_train.extend(data_train_o)
data_test.extend(data_test_o)
data_train.extend(data_train_s)
data_test.extend(data_test_s)
data_train_label = []
data_test_label = []

In [ ]:
for i in range(len(data_train_o)):
    d = []
    for j in range(SAMPLES):
        d.append([1, 0])
    data_train_label.append(d)

In [ ]:
for i in range(len(data_train_s)):
    d = []
    for j in range(SAMPLES):
        d.append([0, 1])
    data_train_label.append(d)

In [ ]:
for i in range(len(data_test_o)):
    d = []
    for j in range(SAMPLES):
        d.append([1, 0])
    data_test_label.append(d)

In [ ]:
for i in range(len(data_test_s)):
    d = []
    for j in range(SAMPLES):
        d.append([0, 1])
    data_test_label.append(d)

In [ ]:
data_train = np.asarray(data_train)
data_test = np.asarray(data_test)
data_train_label = np.asarray(data_train_label)
data_test_label = np.asarray(data_test_label)
pprint.pprint("Train data: " + str(data_train.shape))
pprint.pprint("Train label: " + str(data_train_label.shape))
pprint.pprint("Test data: " + str(data_test.shape))
pprint.pprint("Test label: " + str(data_test_label.shape))

In [ ]:
model = Sequential()

model.add(LSTM(32, return_sequences = True, input_shape = (200, 6)))
model.add(LSTM(16, return_sequences = True))
model.add(Dense(2, activation = softmax))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(x = data_train, y = data_train_label, validation_data = (data_test, data_test_label), epochs = 100, batch_size = 1, verbose = 2)
model.save('model.h5')

data_test_predict = model.predict(data_test)
output = np.argmax(data_test_predict[:, -1, :], axis = 1)
pprint.pprint("out")
pprint.pprint(output)